<a href="https://colab.research.google.com/github/arindamkeswani/RePlicate/blob/main/RePlicate_(HPC_Project).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files 
uploaded = files.upload()

Saving fatma.txt to fatma.txt
Saving image.png to image.png
Saving john.txt to john.txt
Saving juma.txt to juma.txt


In [1]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

In [2]:
student_files = [doc for doc in os.listdir() if doc.endswith('.txt')] #store all text files
student_notes =[open(File).read() for File in  student_files] #stores all lines of all files

In [73]:
vectorize = lambda Text: TfidfVectorizer().fit_transform(Text).toarray()  #to vectorize the words of text files
similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2]) #to store similarity of two documents

In [116]:
vectors = vectorize(student_notes) #store vectorized values
s_vectors = list(zip(student_files, vectors)) #store it with file names
plagiarism_results = set() #to store results in a set
# plagiarism_results =[]


In [129]:
def check_plagiarism(s_vectors_partial):
    print("Starting process...")
    global s_vectors
    for student_a, text_vector_a in s_vectors_partial:  #traverse through students and their vectors (for first document)
        print(f"Started testing:{student_a}")
        new_vectors = s_vectors.copy() 
        
        # current_index = new_vectors.index((student_a, text_vector_a))
        # del new_vectors[current_index]
        

        for student_b , text_vector_b in new_vectors: #traverse through students and their vectors (for first document)
            print(f"Testing {student_a} against {student_b}")
            sim_score = similarity(text_vector_a, text_vector_b)[0][1] #calculate similarity of both documents
            student_pair = sorted((student_a, student_b)) 
            score = (student_pair[0], student_pair[1],sim_score)
            # score = [student_pair[0], student_pair[1],sim_score]
            plagiarism_results.add(score) #add score with file names into the set
            # plagiarism_results.append(score)
        print()
    return plagiarism_results

In [130]:
#Serial
start=time.time()
ans=check_plagiarism(s_vectors)

for data in ans:
    print(data)
end=time.time()
print()
print("Time taken:", end-start)

Starting process...
Started testing:juma.txt
Testing juma.txt against juma.txt
Testing juma.txt against fatma.txt
Testing juma.txt against john.txt

Started testing:fatma.txt
Testing fatma.txt against juma.txt
Testing fatma.txt against fatma.txt
Testing fatma.txt against john.txt

Started testing:john.txt
Testing john.txt against juma.txt
Testing john.txt against fatma.txt
Testing john.txt against john.txt

('fatma.txt', 'john.txt', 0.14806887549598566)
('john.txt', 'juma.txt', 0.5465972177348937)
('juma.txt', 'juma.txt', 1.0000000000000004)
('fatma.txt', 'fatma.txt', 1.0)
('john.txt', 'john.txt', 1.0)
('fatma.txt', 'juma.txt', 0.18643448370323362)

Time taken: 0.004704713821411133


In [134]:
#Parallel [Manual] Part 1
start=time.time()
ans=check_plagiarism(s_vectors[:len(s_vectors)//2])

for data in ans:
    print(data)
end=time.time()
print()
print("Time taken:", end-start)

Starting process...
Started testing:juma.txt
Testing juma.txt against juma.txt
Testing juma.txt against fatma.txt
Testing juma.txt against john.txt

('fatma.txt', 'john.txt', 0.14806887549598566)
('john.txt', 'juma.txt', 0.5465972177348937)
('juma.txt', 'juma.txt', 1.0000000000000004)
('fatma.txt', 'fatma.txt', 1.0)
('john.txt', 'john.txt', 1.0)
('fatma.txt', 'juma.txt', 0.18643448370323362)

Time taken: 0.008398771286010742


In [132]:
#Parallel [Manual] Part 2
start=time.time()
ans=check_plagiarism(s_vectors[len(s_vectors)//2:])

for data in ans:
    print(data)
end=time.time()
print()
print("Time taken:", end-start)

Starting process...
Started testing:fatma.txt
Testing fatma.txt against juma.txt
Testing fatma.txt against fatma.txt
Testing fatma.txt against john.txt

Started testing:john.txt
Testing john.txt against juma.txt
Testing john.txt against fatma.txt
Testing john.txt against john.txt

('fatma.txt', 'john.txt', 0.14806887549598566)
('john.txt', 'juma.txt', 0.5465972177348937)
('juma.txt', 'juma.txt', 1.0000000000000004)
('fatma.txt', 'fatma.txt', 1.0)
('john.txt', 'john.txt', 1.0)
('fatma.txt', 'juma.txt', 0.18643448370323362)

Time taken: 0.0029540061950683594


Ways to achieve data parallelism:
1. Divide s_vectors in parts (more likely)
2. Divide s_vectors[0][1] in parts (potential)

In [ ]:
s_vectors[:len(s_vectors)//2]

In [133]:
#Multiprocessing approach
import multiprocessing
import os
start=time.time()
# def worker1(func1): 
#   func1(m)
  

start=time.time()

pool = multiprocessing.Pool(processes=2) 



l1 = s_vectors[:len(s_vectors)//2]
l2 = s_vectors[len(s_vectors)//2:]


result = pool.map(check_plagiarism, [l1,l2])

for i in result:
  print(i)

# print(ans)

end=time.time()

# ty.append("Multiprocessing [50000 elements]")
# t.append(end-start)

print("Time taken: ",end-start)

Starting process...
Starting process...
Started testing:juma.txt
Testing juma.txt against juma.txt
Started testing:fatma.txt
Testing juma.txt against fatma.txt
Testing fatma.txt against juma.txt
Testing juma.txt against john.txt
Testing fatma.txt against fatma.txt
Testing fatma.txt against john.txt


Started testing:john.txt
Testing john.txt against juma.txt
Testing john.txt against fatma.txt
Testing john.txt against john.txt

{('fatma.txt', 'john.txt', 0.14806887549598566), ('john.txt', 'juma.txt', 0.5465972177348937), ('john.txt', 'john.txt', 1.0), ('juma.txt', 'juma.txt', 1.0000000000000004), ('fatma.txt', 'fatma.txt', 1.0), ('fatma.txt', 'juma.txt', 0.18643448370323362)}
{('fatma.txt', 'john.txt', 0.14806887549598566), ('john.txt', 'juma.txt', 0.5465972177348937), ('john.txt', 'john.txt', 1.0), ('juma.txt', 'juma.txt', 1.0000000000000004), ('fatma.txt', 'fatma.txt', 1.0), ('fatma.txt', 'juma.txt', 0.18643448370323362)}
0.0993509292602539
